# Decorator

## 학습목표
 - 함수 revisited
 - 중첩함수의 이해
 - decorator 이해
 - decorator 구현 실습 

## function
 - python에서의 함수는 1st citizen, 즉 객체로 존재함
 - 객체이기 때문에, 변수 대입, 함수 파라미터 전달 등이 가능

In [1]:
def hello():
    print 'say hello'
    

# 함수를 변수에 대입
hi = hello
print type(hi)
print hi

hi()

<type 'function'>
<function hello at 0x10bb64668>
say hello


In [2]:
def hello(word):
    print word
    
def call_hello(fn, args):
    fn(args)
    

# 함수가 다른 함수의 파라미터로 전달
call_hello(hello, 'say hello world')

say hello world


## 중첩함수 (Nested function)
 - 함수 내부에 다른 함수를 정의 가능
 - 내부에서 정의된 함수를 중첩함수라고 함
   - 중첩함수는 해당 함수가 정의된 함수 내에서 호출 및 반환 가능

In [1]:
def outer_func():
    print 'outer_func call'
    
    # 중첩 함수의 정의
    def inner_func():
        return 'innter_func'
    
    # 중첩 함수 호출 
    print inner_func()
    
outer_func()

outer_func call
innter_func


In [2]:
# 중첩함수의 경우 외부에서 호출 불가
inner_func()

NameError: name 'inner_func' is not defined

In [5]:
def outer_func():
    def inner_func():
        return 'inner_func'
    return inner_func

fn = outer_func()

print fn
print fn()

<function inner_func at 0x10e72f938>
inner_func


In [7]:
def outer_func(num):
    # 중첩 함수에서 외부 함수의 변수에 접근 가능
    def inner_func():
        print num
        return 'inner_func'
    
    return inner_func

fn = outer_func(10)
print fn()

10
inner_func


## Closure or Closure function
 - enclosing scope(외부 함수)의 변수가 소멸되더라도 해당 변수의 값을 기억하고 사용할 수 있는 함수
 - 위의 예제에서 inner_func은 closure
 - outer_func이 이미 호출 종료되어 num의 scope이 소멸되었는데도 해당 값을 기억
 
 - closure의 사용
   - class를 사용하지 않고 객체지향적인 솔루션을 제공 
   - 일반적으로 제공해야할 기능(method)이 적은 경우, closure를 사용하여 기능을 제공함
   - 그 이외의 경우 class를 사용하여 구현

In [8]:
# closure를 사용할 수 도 있음
def get_power_of(n):
    def power(x):
        return x ** n
    return power


power5 = get_power_of(5)
power8 = get_power_of(8)

print power5(2)
print power8(2)

32
256


## Decorator
 - 함수에 기능을 추가하여 다시 그 함수를 반환하는 함수
 - Closure function을 활용
 - https://www.python.org/dev/peps/pep-0318/
 - 고급 파이썬 개발자로 가는 길목 중 하나

In [8]:
# decorator 함수 정의
def decorate_func(fn):
    def inner():
        print 'decoration added'
        fn()
    return inner

def simple():
    print 'simple function'
    
# simple 함수에 기능을 추가한(decorate 한) decorated 함수
decorated = decorate_func(simple)

# 결과가 decoration됨!
decorated()

decoration added
simple function


In [7]:
# 일반 함수 호출
simple()

simple function


In [ ]:
decorated = decorate_func(simple)
decorated()

In [9]:
# 보통 구현 시, 새로운 변수를 생성하지 않고 원래 함수에 재할당하여 사용
simple = decorate_func(simple)

# 결과가 decoration됨!
simple()

decoration added
simple function


* decorator 사용 이유
 - 함수, 혹은 메쏘드에 새로운 기능을 추가
 - 그렇다면 왜 소스 코드를 수정하지 않는가?
   - 여러 함수에 동일한 기능을 추가할 수도 있음
   - 예를들어, 모든 함수에 전달된 파라미터의 유효성 검사가 필요하다고 가정
   - 유효성 검사 코드가 각 함수마다 복사되면 수정의 어려움이 존재

In [13]:
# decorator 함수 정의
def decorate_func(fn):
    def inner():
        print 'decoration added'
        fn()
    return inner

def simple():
    print 'simple function'
    
# 사실 귀찮고, python 답지 않은 문법임
simple = decorate_func(simple)
simple()

decoration added
simple function


* @ 심볼 사용
 - decorator를 생성하기 위한 syntactic sugar (문법적인 편의성)

In [14]:
# decorator 함수 정의
def decorate_func(fn):
    def inner():
        print 'decoration added'
        fn()
    return inner

# 아래와 같이 @를 사용하여 decoration 가능!
@decorate_func
def simple():
    print 'simple function'
    
# 결과가 decorated됨!
simple()

decoration added
simple function


## 파라미터가 있는 함수 Decorator

In [15]:
def divide(a, b):
    return a / b

In [16]:
divide(3, 2)

1

In [10]:
divide(3, 0)

NameError: name 'divide' is not defined

* 중첩함수에 꾸미고자 하는 함수와 동일하게 파라미터를 가져가면 됨

In [11]:
def decorate_divide(fn):
    def wrapper(a, b):
        if b == 0:
            print 'zero cannot be divided!'
            return 
        return fn(a, b)
    return wrapper

def divide(a, b):
    return a / b


In [12]:
divide = decorate_divide(divide)
print divide(9, 3)
print divide(9, 0)

3
zero cannot be divided!
None


In [14]:
def decorate_divide(fn):
    def wrapper(a, b):
        if b == 0:
            print 'zero cannot be divided!'
            return 
        return fn(a, b)
    return wrapper

# 추후, 아래와 같이 사용할 것!
@decorate_divide
def divide(a, b):
    return a / b

In [15]:
print divide(9, 0)
print divide(81, 7)

zero cannot be divided!
None
11


## 모든 함수에 대한 Decorator

In [16]:
def general_decorator(fn):
    def wrapper(*args, **kwargs):
        print('function is decorated..')
        return fn(*args, **kwargs)
    return wrapper


@general_decorator
def add(a, b):
    return a + b

@general_decorator
def print_hello():
    print 'hello'
    
    
print add(4, 5)
print_hello()

function is decorated..
9
function is decorated..
hello


## Decorator Chaining
 - 복수개의 decorator 적용 가능
 - decorated된 순서가 중요

In [17]:
def star(fn):
    def wrapper(*args, **kwargs):
        print('function is decorated with ******************')
        return fn(*args, **kwargs)
    return wrapper

def at(fn):
    def wrapper(*args, **kwargs):
        print('function is decorated with @@@@@@@@@@@@@@@@@@')
        return fn(*args, **kwargs)
    return wrapper



In [18]:
@at
@star
def print_hello():
    print 'hello'
    
print_hello()

function is decorated with @@@@@@@@@@@@@@@@@@
function is decorated with ******************
hello


In [19]:
@star
@at
def print_hello():
    print 'hello'
    
print_hello()

function is decorated with ******************
function is decorated with @@@@@@@@@@@@@@@@@@
hello


## Method decoration
 - 객체의 method도 decorating 가능
 - 객체의 경우 중첩 함수에서 최초의 파라미터를 self를 추가

In [21]:
print '{} {}'.format(10, 100)
print '{0} {0} {0} {1}'.format(10, 100)
print '{1} {0}'.format(10, 100)

print '{aa} {bb}'.format(aa = 'aaaa', bb = 'cccc')

10 100
10 10 10 100
100 10
aaaa cccc


In [20]:
def h1_tag(func):
    def func_wrapper(self, *args, **kwargs):
        return "<h1>{0}</h1>".format(func(self, *args, **kwargs))
    return func_wrapper

class Person(object):
    def __init__(self):
        self.firt_name = 'Aaron'
        self.last_name = 'Byun'

    @h1_tag
    def get_name(self):
        return self.firt_name + ' ' + self.last_name
    
    @h1_tag
    def get_x(self, x):
        return x * 2

aaron = Person()
print aaron.get_x('Ho')

<h1>Aaron Byun</h1>
<h1>HoHo</h1>


## Decorator with parameters
 - decorator에 파라미터를 추가 가능

In [23]:
def h1_tag(func):
    def func_wrapper(self, *args, **kwargs):
        return "<h1>{0}</h1>".format(func(self, *args, **kwargs))
    return func_wrapper

print_hello = h1_tag(print_hello)

# 중첩 함수의 뎁스를 하나 더 두어서 생성
def star(star_num=20):
    def callable(fn):
        def wrapper(*args, **kwargs):
            print 'function is decorated with {}'.format('*' * star_num )
            return fn(*args, **kwargs)
        return wrapper
    return callable

def print_hello():
    print 'hello'

print_hello = star(5)(print_hello)
print_hello()



function is decorated with *****
hello


In [25]:
#위의 코드를 parameter가 있는 decorator화 할 수 있다
@star(star_num=40)
def print_hello():
    print 'hello'

    
print_hello()

function is decorated with ****************************************
hello


 * 연습문제)
  1. 함수 파라미터의 타입과 값을 출력하는 decorator를 작성하시오
  2. 아래의 두 함수가 있다. 이때 원래의 함수 호출 시, deprecated 되었다고 출력하고 새로운 함수를 출력하도록 작성하시오

In [35]:
def func_old():
    print 'old function'
    
def func_new():
    print 'new function'

False
